## Homework 3: Classification

In this homework, we will use the California Housing Prices data from Kaggle.

In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# download dataset from internet

#!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

In [3]:
# read the data into pandas dataframe
df_orig = pd.read_csv("housing.csv")
df_orig.head(n=2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY


## Data Preparation

In [4]:
# use only the following features
features_list= ['latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value',
'ocean_proximity']
df = df_orig[ features_list ]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   latitude            20640 non-null  float64
 1   longitude           20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [5]:
df.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [6]:
# fill in missing values with value 0
df.total_bedrooms = df.total_bedrooms.fillna(0)
df.isnull().sum()

latitude              0
longitude             0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

* Create a new column _rooms_per_household_ by dividing the column _total_rooms_ by the column _households_ from dataframe.
* Create a new column _bedrooms_per_room_ by dividing the column _total_bedrooms_ by the column _total_rooms_ from dataframe.
* Create a new column _population_per_household_ by dividing the column _population_ by the column _households_ from dataframe.


In [7]:
df['rooms_per_household'] = df.total_rooms/df.households
df['bedrooms_per_room'] = df.total_bedrooms/df.total_rooms
df['population_per_household']= df.population/df.households
df.isnull().sum()

latitude                    0
longitude                   0
housing_median_age          0
total_rooms                 0
total_bedrooms              0
population                  0
households                  0
median_income               0
median_house_value          0
ocean_proximity             0
rooms_per_household         0
bedrooms_per_room           0
population_per_household    0
dtype: int64

## Question 1

What is the most frequent observation (mode) for the column _ocean_proximity_?

Options:

* NEAR BAY
* <1H OCEAN
* INLAND
* NEAR OCEAN

In [8]:
df.ocean_proximity.mode()

0    <1H OCEAN
Name: ocean_proximity, dtype: object

In [9]:
# Answer: mode is <1H OCEAN; let's double check
df.ocean_proximity.value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

## Split the Data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value (median_house_value) is not in your dataframe.(???)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [12]:
len(df_train), len(df_val), len(df_test)

(12384, 4128, 4128)

In [13]:
# reset index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [14]:
y_train = df_train.median_house_value
y_val = df_val.median_house_value
y_test = df_test.median_house_value

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

## Question 2  

Create the correlation matrix for the numerical features of your train dataset. 

In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.  

What are the two features that have the biggest correlation in this dataset?

Options:
* _total_bedrooms_ and _households_
* _total_bedrooms_ and _total_rooms_
* _population_ and _households_
* _population_per_household_ and _total_rooms_

In [15]:
numerical_features = list( df_train.dtypes[ df_train.dtypes != 'object' ].index )
numerical_features

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [16]:
df_train_numerical = df_train[ numerical_features ]
df_train_numerical.head(n=3)

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
0,34.43,-119.67,39.0,1467.0,381.0,1404.0,374.0,2.3681,3.922460,0.259714,3.754011
1,33.74,-118.32,24.0,6097.0,794.0,2248.0,806.0,10.1357,7.564516,0.130228,2.789082
2,39.13,-121.62,41.0,1317.0,309.0,856.0,337.0,1.6719,3.908012,0.234624,2.540059


In [17]:
df_train_numerical.corr()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
latitude,1.000000,-0.925005,0.002477,-0.025914,-0.059730,-0.100272,-0.063529,-0.076805,0.119118,-0.124507,-0.002301
longitude,-0.925005,1.000000,-0.099812,0.036449,0.063840,0.091670,0.049762,-0.016426,-0.034814,0.102320,0.011022
housing_median_age,0.002477,-0.099812,1.000000,-0.363522,-0.324156,-0.292476,-0.306119,-0.119591,-0.181275,0.129456,0.012167
total_rooms,-0.025914,0.036449,-0.363522,1.000000,0.931546,0.853219,0.921441,0.198951,0.168926,-0.194185,-0.029452
total_bedrooms,-0.059730,0.063840,-0.324156,0.931546,1.000000,0.877340,0.979399,-0.009833,0.010381,0.078094,-0.034301
population,-0.100272,0.091670,-0.292476,0.853219,0.877340,1.000000,0.906841,-0.000849,-0.076210,0.031592,0.064998
households,-0.063529,0.049762,-0.306119,0.921441,0.979399,0.906841,1.000000,0.011925,-0.085832,0.058004,-0.032522
median_income,-0.076805,-0.016426,-0.119591,0.198951,-0.009833,-0.000849,0.011925,1.000000,0.394154,-0.616617,-0.000454
rooms_per_household,0.119118,-0.034814,-0.181275,0.168926,0.010381,-0.076210,-0.085832,0.394154,1.000000,-0.500589,0.001801
bedrooms_per_room,-0.124507,0.102320,0.129456,-0.194185,0.078094,0.031592,0.058004,-0.616617,-0.500589,1.000000,-0.002851


In [18]:
corr = df_train_numerical.corr().abs().unstack()
sorted = corr.sort_values(ascending=False)
sorted[:20]

latitude                  latitude                    1.000000
longitude                 longitude                   1.000000
bedrooms_per_room         bedrooms_per_room           1.000000
rooms_per_household       rooms_per_household         1.000000
median_income             median_income               1.000000
households                households                  1.000000
total_bedrooms            total_bedrooms              1.000000
total_rooms               total_rooms                 1.000000
housing_median_age        housing_median_age          1.000000
population                population                  1.000000
population_per_household  population_per_household    1.000000
households                total_bedrooms              0.979399
total_bedrooms            households                  0.979399
total_rooms               total_bedrooms              0.931546
total_bedrooms            total_rooms                 0.931546
longitude                 latitude                    0

In [19]:
#  two features that have the biggest correlation are: households and total_bedrooms

## Make median_house_value binary 

We need to turn the median_house_value variable from numeric into binary. 

Let's create a variable _above_average_ which is 1 if the _median_house_value_ is above its mean value and 0 otherwise.

### Question 3

Calculate the mutual information score with the (binarized) price for the categorical variable that we have. **Use the training set only.**  

What is the value of mutual information? 

Round it to 2 decimal digits using round(score, 2)
Options:
* 0.26
* 0
* 0.10
* 0.16

In [20]:
y_train_mean = y_train.mean()  # astype(int)
above_average_train = ( y_train > y_train_mean ).astype(int)
above_average_train

0        1
1        1
2        0
3        1
4        1
        ..
12379    0
12380    0
12381    1
12382    0
12383    0
Name: median_house_value, Length: 12384, dtype: int64

In [21]:
from sklearn.metrics import mutual_info_score

In [22]:
mutual_info_score(above_average_train, df_train.ocean_proximity)

0.10138385763624205

In [23]:
# Answer: The mutual information score is 0.10 (rounded to 2 dec. places). 

### Question 4

Now let's train a logistic regression. 

Remember that we have one categorical variable _ocean_proximity_ in the data. Include it using one-hot encoding.  

Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42) 

Calculate the accuracy on the validation dataset and round it to 2 decimal digits.  

Options:
* 0.60
* 0.72
* 0.84
* 0.95

In [24]:
# recap which features are numerical
numerical_features

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [25]:
categorical_features = ['ocean_proximity']

In [26]:
from sklearn.feature_extraction import DictVectorizer

In [27]:
# carry out the one-hot encoding of features on training and validation sets

dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [28]:
# Get the y variables to be categorical
y_train_cat = ( y_train > y_train_mean ).astype(int)
y_val_cat = ( y_val > y_train_mean ).astype(int)
y_test_cat = ( y_test > y_train_mean ).astype(int)

In [29]:
# Fit a logistic regression model on the training set

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train_cat)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [30]:
# Run the model prediction using the validation set
y_val_predict = model.predict(X_val)

In [31]:
# Compute accuracy of the model prediction on the validation set.  Answer: accuracy = 0.84
accuracy = (y_val_predict == y_val_cat).mean()
round(accuracy, 2)

0.84

### Question 5 

Let's find the least useful feature using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4). 

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?
* _total_rooms_
* _total_bedrooms_
* _population_
* _households_ 

note: the difference doesn't have to be positive

In [32]:
selected_features = ['total_rooms', 'total_bedrooms', 'population','households']
#df_train.drop('total_rooms', axis=1)

In [33]:
acc_list = []
for sf in selected_features:
    # drop the feature from the training and validation datasets
    df_train_drop = df_train.drop(sf, axis=1)
    df_val_drop = df_val.drop(sf, axis=1)
    
    # one-hot encoding
    dv = DictVectorizer(sparse=False)
    
    train_dict = df_train_drop.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = df_val_drop.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    # fit model on smaller training set
    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train_cat)
    
    # predictions on validation set
    y_val_predict = model.predict(X_val)
    
    # validation set accuracy
    acc = (y_val_predict == y_val_cat).mean()
    
    acc_list.append(acc)
    
acc_list    

[0.8364825581395349,
 0.8369670542635659,
 0.8263081395348837,
 0.8338178294573644]

In [34]:
df_accuracy = pd.DataFrame(acc_list, index=selected_features)
df_accuracy

,0
total_rooms,0.836483
total_bedrooms,0.836967
population,0.826308
households,0.833818


In [35]:

df_accuracy.columns=['accuracy']


In [36]:
df_accuracy['diff']=  accuracy - df_accuracy['accuracy']
df_accuracy

,accuracy,diff
total_rooms,0.836483,0.000242
total_bedrooms,0.836967,-0.000242
population,0.826308,0.010417
households,0.833818,0.002907


In [37]:
# Answer: total_bedrooms
# Although total_rooms and total_bedrooms have the same absolute difference from accuracy of the full model, 
# I pick total_bedrooms because dropping it causes an increase in accuracy over the full model => it is less useful
# than total_rooms (which actually leads to a gain in accuracy when it is added to the model)

### Question 6

For this question, we'll see how to use a linear regression model from Scikit-Learn
We'll need to use the original column 'median_house_value'. 
Apply the logarithmic transformation to this column.
Fit the Ridge regression model (model = Ridge(alpha=a, solver="sag", random_state=42)) on the training data.
This model has a parameter alpha. Let's try the following values: \[0, 0.01, 0.1, 1, 10\] 

Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.
If there are multiple options, select the smallest alpha.

Options:

* 0
* 0.01
* 0.1
* 1
* 10

In [50]:
# Apply log transformation to response variable
y_train_log = np.log1p(y_train.values)
y_val_log = np.log1p(y_val.values)
y_test_log = np.log1p(y_test.values)

In [51]:
# Prepare the feature matrix 

dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [52]:
from sklearn.linear_model import Ridge

def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [53]:
alpha_list = [0, 0.01, 0.1, 1, 10]
rmse_dict = {}

for val in alpha_list:
    # Fit ridge regression model on training data using alpha=val
    model = Ridge(alpha=val, solver="sag", random_state=42)
    model.fit(X_train, y_train_log)
    # Predict on the validation set
    y_val_pred = model.predict(X_val)
    # Compute rmse
    rmse_dict[val]= round( rmse(y_val_pred, y_val_log), 3)
    
rmse_dict

{0: 0.524, 0.01: 0.524, 0.1: 0.524, 1: 0.524, 10: 0.524}

In [42]:
# Answer: alpha=0 as it is the smallest alpha and all alpha values lead to same value of rmse

In [49]:
X_train.shape


(12384, 16)